# Algoritmo Genético para o Problema da Mochila
Este Notebook demonstra a implementação de um Algoritmo Genético (AG) para resolver o Problema da Mochila (Knapsack Problem). 

O objetivo é selecionar um conjunto de itens, cada um com um peso e um valor, de forma a maximizar o valor total carregado em uma mochila com capacidade limitada.

## 1. O Problema da Mochila
O Problema da Mochila é um problema clássico de otimização combinatória. Dado um conjunto de itens, cada um com um peso e um valor associado, e uma mochila com uma capacidade máxima de peso, o objetivo é determinar quais itens incluir na mochila de forma que:

O peso total dos itens selecionados não exceda a capacidade da mochila.
O valor total dos itens selecionados seja o máximo possível.

In [3]:
import random
from typing import List, Tuple, Optional

In [4]:
# Repsesentação dos items
class Item:
    def __init__(self, name: str, weight: int, value: int):
        self.name = name
        self.weight = weight
        self.value = value

    def __repr__(self) -> str:
        return f"Item(nome='{self.name}', peso={self.weight}, valor={self.value})"

In [5]:
# Criação e Inicialização da População
def criar_individuo(num_items: int) -> List[int]:
    """Cria um indivíduo aleatório (cromossomo) como uma lista de bits."""
    return [random.randint(0, 1) for _ in range(num_items)]

def inicializar_populacao(tam_populacao: int, num_items: int) -> List[List[int]]:
    """Inicializa uma população de indivíduos aleatórios."""
    return [criar_individuo(num_items) for _ in range(tam_populacao)]

In [6]:
def calcular_detalhes_individuo(individuo: List[int], items: List[Item], capacidade_maxima: int) -> Tuple[int, int, int]:
    """
    Calcula os detalhes de um indivíduo (solução).
    Retorna: (pontuacao_fitness, valor_total_real, peso_total_real)
    A pontuacao_fitness é o valor_total_real se o peso estiver dentro da capacidade,
    caso contrário, é 0 (penalidade).
    """
    peso_total = 0
    valor_total = 0
    for i, gene in enumerate(individuo):
        if gene == 1:
            peso_total += items[i].weight
            valor_total += items[i].value

    if peso_total > capacidade_maxima:
        pontuacao_fitness = 0  # Penaliza fortemente soluções inválidas
    else:
        pontuacao_fitness = valor_total
    return pontuacao_fitness, valor_total, peso_total

In [7]:
# Seleção

def selecionar_pai_torneio(populacao: List[List[int]],
                             pontuacoes_fitness: List[int],
                             tam_torneio: int) -> List[int]:
    """Seleciona um único pai usando seleção por torneio."""
    if not populacao:
        raise ValueError("A população não pode estar vazia para seleção por torneio.")
    
    # Garante que o tamanho do torneio não exceda o tamanho da população disponível
    k = min(tam_torneio, len(populacao))
    if k == 0: # Se a população efetivamente se tornou vazia (não deveria, mas como defesa)
        raise ValueError("Não é possível selecionar de uma população vazia ou com tamanho de torneio 0.")


    indices_torneio = random.sample(range(len(populacao)), k)

    melhor_indice_contendor = -1
    melhor_fitness_contendor = -1 # Assume-se fitness não negativo

    for indice in indices_torneio:
        if pontuacoes_fitness[indice] > melhor_fitness_contendor:
            melhor_fitness_contendor = pontuacoes_fitness[indice]
            melhor_indice_contendor = indice
            
    # Se todos os contendores tiverem fitness 0 (ou negativo, se fosse o caso),
    # e nenhum melhor foi achado (melhor_indice_contendor ainda é -1),
    # ou se melhor_indice_contendor for válido.
    if melhor_indice_contendor == -1 and indices_torneio: # Todos os fitness são iguais (ex: 0)
        return populacao[indices_torneio[0]] # Retorna o primeiro do torneio
        
    return populacao[melhor_indice_contendor]

In [8]:
# Cruzamento

def cruzamento_ponto_unico(pai1: List[int], pai2: List[int], taxa_cruzamento: float) -> Tuple[List[int], List[int]]:
    """Realiza cruzamento de ponto único entre dois pais se a taxa_cruzamento for atingida."""
    filho1, filho2 = pai1[:], pai2[:] # Copia os pais por padrão
    if random.random() < taxa_cruzamento and len(pai1) > 1 and len(pai2) > 1: # Garante que cruzamento é possível
        ponto = random.randint(1, len(pai1) - 1)
        filho1 = pai1[:ponto] + pai2[ponto:]
        filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2

In [9]:
# Mutação

def mutacao_bit_flip(individuo: List[int], taxa_mutacao: float) -> List[int]:
    """Realiza mutação bit-flip em um indivíduo."""
    individuo_mutado = individuo[:]
    for i in range(len(individuo_mutado)):
        if random.random() < taxa_mutacao:
            individuo_mutado[i] = 1 - individuo_mutado[i]  # Inverte o bit
    return individuo_mutado

In [10]:
# Algoritmo Genético 

def algoritmo_genetico_mochila(
    items_data: List[Tuple[str, int, int]],
    capacidade_maxima: int,
    tam_populacao: int = 100,
    num_geracoes: int = 200,
    taxa_mutacao: float = 0.02,
    taxa_cruzamento: float = 0.85,
    contagem_elitismo: int = 2,
    tam_torneio: int = 5,
    seed: Optional[int] = None,
    verbose: bool = True # Adicionado para controlar a verbosidade
) -> Tuple[Optional[List[int]], int, int]:
    """
    Resolve o problema da mochila usando um algoritmo genético.
    Retorna a melhor solução, seu valor e seu peso.
    """
    if seed is not None:
        random.seed(seed)

    items = [Item(nome, peso, valor) for nome, peso, valor in items_data]
    num_items = len(items)

    if num_items == 0:
        if verbose: print("Nenhum item fornecido.")
        return None, 0, 0
    if tam_populacao <=0:
        if verbose: print("Tamanho da população deve ser positivo.")
        return None, 0, 0
    if contagem_elitismo < 0:
        contagem_elitismo = 0
    if contagem_elitismo >= tam_populacao:
        if verbose: print("Contagem de elitismo deve ser menor que o tamanho da população. Ajustando...")
        contagem_elitismo = max(0, tam_populacao -1) 
    if tam_torneio <= 0:
        if verbose: print("Tamanho do torneio deve ser positivo. Usando 1.")
        tam_torneio = 1


    if verbose:
        print(f"--- Algoritmo Genético para o Problema da Mochila ---")
        print(f"Itens: {num_items}, Capacidade Máxima: {capacidade_maxima}")
        print(f"Tamanho População: {tam_populacao}, Gerações: {num_geracoes}")
        print(f"Taxa Mutação: {taxa_mutacao}, Taxa Cruzamento: {taxa_cruzamento}")
        print(f"Elitismo: {contagem_elitismo}, Tamanho Torneio: {tam_torneio}\n")

    populacao_atual = inicializar_populacao(tam_populacao, num_items)
    
    melhor_solucao_geral: Optional[List[int]] = None
    melhor_fitness_geral = -1
    melhor_peso_geral = -1

    for geracao in range(num_geracoes):
        avaliacoes_populacao = []
        fitness_scores_populacao_atual = [] 

        for individuo in populacao_atual:
            fitness, valor_real, peso_real = calcular_detalhes_individuo(individuo, items, capacidade_maxima)
            avaliacoes_populacao.append((fitness, valor_real, peso_real, individuo))
            fitness_scores_populacao_atual.append(fitness)
        
        avaliacoes_populacao.sort(key=lambda x: x[0], reverse=True)

        if avaliacoes_populacao: # Checa se a lista não está vazia
            melhor_fitness_geracao_atual = avaliacoes_populacao[0][0]
            melhor_peso_geracao_atual = avaliacoes_populacao[0][2]
            melhor_individuo_geracao_atual = avaliacoes_populacao[0][3]

            if melhor_fitness_geracao_atual > melhor_fitness_geral:
                melhor_fitness_geral = melhor_fitness_geracao_atual
                melhor_solucao_geral = melhor_individuo_geracao_atual
                melhor_peso_geral = melhor_peso_geracao_atual
                if verbose:
                    print(f"Geração {geracao + 1}: Nova melhor solução! Fitness (Valor) = {melhor_fitness_geral}, Peso = {melhor_peso_geral}")
            elif verbose and (geracao + 1) % 20 == 0:
                print(f"Geração {geracao + 1}: Melhor Fitness da Geração = {melhor_fitness_geracao_atual}, Peso = {melhor_peso_geracao_atual}. Melhor Geral: {melhor_fitness_geral}")
        elif verbose: # População avaliada está vazia, o que não deveria acontecer com tam_populacao > 0
            print(f"Geração {geracao + 1}: População avaliada vazia. Melhor Geral: {melhor_fitness_geral}")


        nova_populacao = []
        for i in range(min(contagem_elitismo, len(avaliacoes_populacao))):
            nova_populacao.append(avaliacoes_populacao[i][3])

        num_descendentes_necessarios = tam_populacao - len(nova_populacao)
        descendentes_gerados = 0
        
        # Loop para gerar descendentes até preencher a nova população
        # Adiciona verificações para evitar erros com populações pequenas ou vazias
        while descendentes_gerados < num_descendentes_necessarios:
            if not populacao_atual or not fitness_scores_populacao_atual or len(populacao_atual) < max(1,tam_torneio) : 
                # Se a população atual é inadequada para seleção, preenche com aleatórios ou para
                if verbose and descendentes_gerados < num_descendentes_necessarios:
                     print(f"População atual ({len(populacao_atual)}) pequena demais para seleção (torneio {tam_torneio}). Preenchendo com aleatórios se necessário.")
                # Preenche o restante com indivíduos aleatórios para manter o tamanho da população
                while len(nova_populacao) < tam_populacao:
                    nova_populacao.append(criar_individuo(num_items))
                break 

            pai1 = selecionar_pai_torneio(populacao_atual, fitness_scores_populacao_atual, tam_torneio)
            # Garante que haja pelo menos dois pais para cruzar se len(populacao_atual) > 1
            pai2_candidatos = [p for p in populacao_atual if p is not pai1] # Evita selecionar o mesmo pai se possível
            if not pai2_candidatos and len(populacao_atual) > 0: # Se só resta o pai1
                 pai2 = pai1 # Usa o mesmo pai se for o único
            elif not pai2_candidatos: # Se população atual está vazia após seleção do pai1 (improvável)
                 pai2 = criar_individuo(num_items) # Cria um pai aleatório
            else:
                 pai2_fitness_candidatos = [fitness_scores_populacao_atual[populacao_atual.index(p)] for p in pai2_candidatos]
                 pai2 = selecionar_pai_torneio(pai2_candidatos, pai2_fitness_candidatos, tam_torneio)


            filho1, filho2 = cruzamento_ponto_unico(pai1, pai2, taxa_cruzamento)

            nova_populacao.append(mutacao_bit_flip(filho1, taxa_mutacao))
            descendentes_gerados += 1

            if descendentes_gerados < num_descendentes_necessarios:
                nova_populacao.append(mutacao_bit_flip(filho2, taxa_mutacao))
                descendentes_gerados += 1
        
        populacao_atual = nova_populacao[:tam_populacao]

    if verbose:
        print("\n--- Algoritmo Genético Finalizado ---")
        if melhor_solucao_geral:
            print(f"Melhor solução encontrada: {melhor_solucao_geral}")
            itens_selecionados_nomes = [items[i].name for i, gene in enumerate(melhor_solucao_geral) if gene == 1]
            print(f"Itens selecionados: {itens_selecionados_nomes}")
            print(f"Valor Total: {melhor_fitness_geral}")
            print(f"Peso Total: {melhor_peso_geral}/{capacidade_maxima}")
        else:
            print("Nenhuma solução válida foi encontrada.")

    return melhor_solucao_geral, melhor_fitness_geral, melhor_peso_geral
    

In [11]:
# Exemplo de itens: (nome, peso, valor)
config_itens_ex1 = [
    ("Lanterna", 2, 15), ("Saco de Dormir", 5, 30), ("Comida Enlatada", 10, 50),
    ("Corda", 3, 20), ("Mapa", 1, 10), ("Bússola", 1, 15),
    ("Kit Primeiros Socorros", 4, 25), ("Cantil", 2, 20), ("Faca", 1, 18),
    ("Repelente", 1, 12), ("Câmera", 3, 40), ("Livro", 2, 5),
    ("Barraca", 15, 70), ("Fogareiro", 6, 35), ("Panelas", 4, 22),
    ("Rádio Solar", 3, 28), ("Bateria Extra", 2, 22), ("Chocolate", 1, 16)
]
capacidade_mochila_ex1 = 35

# Executa o algoritmo genético
melhor_solucao, valor_final, peso_final = algoritmo_genetico_mochila(
    items_data=config_itens_ex1,
    capacidade_maxima=capacidade_mochila_ex1,
    tam_populacao=100,
    num_geracoes=150,
    taxa_mutacao=0.03,
    taxa_cruzamento=0.90,
    contagem_elitismo=3,
    tam_torneio=5,
    seed=42, # Para resultados reprodutíveis neste exemplo
    verbose=True
)

print(f"\nResultado final para Exemplo 1:")
print(f"Melhor Solução (cromossomo): {melhor_solucao}")
print(f"Valor Total: {valor_final}")
print(f"Peso Total: {peso_final}/{capacidade_mochila_ex1}")

--- Algoritmo Genético para o Problema da Mochila ---
Itens: 18, Capacidade Máxima: 35
Tamanho População: 100, Gerações: 150
Taxa Mutação: 0.03, Taxa Cruzamento: 0.9
Elitismo: 3, Tamanho Torneio: 5

Geração 1: Nova melhor solução! Fitness (Valor) = 265, Peso = 33
Geração 2: Nova melhor solução! Fitness (Valor) = 268, Peso = 35
Geração 3: Nova melhor solução! Fitness (Valor) = 281, Peso = 34
Geração 5: Nova melhor solução! Fitness (Valor) = 286, Peso = 35
Geração 6: Nova melhor solução! Fitness (Valor) = 298, Peso = 35
Geração 8: Nova melhor solução! Fitness (Valor) = 303, Peso = 35
Geração 18: Nova melhor solução! Fitness (Valor) = 306, Peso = 35
Geração 20: Melhor Fitness da Geração = 306, Peso = 35. Melhor Geral: 306
Geração 40: Melhor Fitness da Geração = 306, Peso = 35. Melhor Geral: 306
Geração 60: Melhor Fitness da Geração = 306, Peso = 35. Melhor Geral: 306
Geração 80: Melhor Fitness da Geração = 306, Peso = 35. Melhor Geral: 306
Geração 100: Melhor Fitness da Geração = 306, Pes